# Trabalho 01 - Rede MLP

Harrison Caetano Candido                                    RA:156264

Implementar uma rede MLP sem utilizar pacotes prontos, como Pytorch, TensorFlow etc. Com a rede implementada, desenvolver dois modelos:
- Classificaçao;
- Regressão.

Avaliar os hiperparâmetros dos modelos variando o número de camadas, número de neurônios e taxas (eta e momentum).

In [ ]:
import numpy as np

class MLP:
  # aqui fazemos a inicializacao de alguns hiperparametros da rede
  def __init__(self, layer_size, learning_rate=0.01, momentum=0):
    self.layer_size = layer_size
    self.learning_rate = learning_rate
    self.momentum = momentum
    self.weights = [np.random.rand(layer_size[i], layer_size[i+1]) - 0.5 for i in range(len(layer_size) - 1)]
    self.biases = [np.random.rand(1, layer_size[i+1]) - 0.5 for i in range(len(layer_size) - 1)]
    self.velocities = [np.zeros_like(w) for w in self.weights]

  # ------------------------------------- funcoes de ativacao -------------------------------------
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))

  def sigmoid_derivative(self, x):
    return self.sigmoid(x) * (1 - self.sigmoid(x))

  def forward(self, x):
    self.activations = [x]
    self.z_values = []

    for w, b in zip(self.weights, self.biases):
        # Corrigir a multiplicação de matrizes
        z = np.dot(self.activations[-1], w) + b
        self.z_values.append(z)
        # Aplicar a função de ativação sigmoid no somatório
        self.activations.append(self.sigmoid(z))

    # Retornar o último elemento da lista de ativação
    return self.activations[-1]


  def backward(self, x, y):
    output_error = y - self.activations[-1]
    deltas = [output_error * self.sigmoid_derivative(self.z_values[-1])]

    # Calcular os deltas para todas as camadas ocultas
    for i in range(len(self.weights) - 1, 0, -1):
        delta = np.dot(deltas[0], self.weights[i].T) * self.sigmoid_derivative(self.z_values[i - 1])
        deltas.insert(0, delta)  # Adicionar delta ao início da lista

    # Atualizar pesos e biases
    for i in range(len(self.weights)):
        gradient = np.dot(self.activations[i].T, deltas[i])
        self.velocities[i] = self.momentum * self.velocities[i] + self.learning_rate * gradient
        self.weights[i] += self.velocities[i]
        self.biases[i] += self.learning_rate * np.sum(deltas[i], axis=0, keepdims=True)

  def train(self, x, y, epochs):
    for e in range(epochs):
      output = self.forward(x)
      self.backward(x, y)
      if e % 100 == 0:
        loss = np.mean(np.square(output - y))
        print(f"Epoch {e}, Loss: {loss}")

def evaluate_hyperparameters(task_type):
  if task_type == "classification":
    # hiperparametros para classificacao
    x = np.array([[0, 0], [0, 1], [1, 0], [1,1]])
    y = np.array([[0], [1], [1], [0]])
  elif task_type == "regression":
    x = np.linspace(-1, 1, 100).reshape(-1, 1)
    y = x**2

  configs = [
      {"layers": [x.shape[1], 4, 1], "learning_rate": 0.1, "momentum": 0},
      {"layers": [x.shape[1], 8, 1], "learning_rate": 0.1, "momentum": 0.9},
      {"layers": [x.shape[1], 4, 4, 1], "learning_rate": 0.05, "momentum": 0.5}
  ]


  for config in configs:
    mlp = MLP(config["layers"], config["learning_rate"], config["momentum"])
    mlp.train(x, y, epochs=1000)

    for sample in x[:5]:
      print(f"Entrada: {sample}, Saída prevista: {mlp.forward(sample)}")



In [ ]:
print('evaluating classification:')
evaluate_hyperparameters("classification")

evaluating classification:
Epoch 0, Loss: 0.25182411305888786
Epoch 100, Loss: 0.2501508508680531
Epoch 200, Loss: 0.2501281607056393
Epoch 300, Loss: 0.25010776355131803
Epoch 400, Loss: 0.2500891001846181
Epoch 500, Loss: 0.2500718337173986
Epoch 600, Loss: 0.2500556828626769
Epoch 700, Loss: 0.2500404088394556
Epoch 800, Loss: 0.2500258052716856
Epoch 900, Loss: 0.25001169024978265
Entrada: [0 0], Saída prevista: [[0.49969199]]
Entrada: [0 1], Saída prevista: [[0.51193677]]
Entrada: [1 0], Saída prevista: [[0.4872861]]
Entrada: [1 1], Saída prevista: [[0.49921764]]
Epoch 0, Loss: 0.29056934235035187
Epoch 100, Loss: 0.24993714847825166
Epoch 200, Loss: 0.24984698585309417
Epoch 300, Loss: 0.24967856774293906
Epoch 400, Loss: 0.24920410123479356
Epoch 500, Loss: 0.2465497787037349
Epoch 600, Loss: 0.21892918082141086
Epoch 700, Loss: 0.14470901411500936
Epoch 800, Loss: 0.06217760660792823
Epoch 900, Loss: 0.02310674123912062
Entrada: [0 0], Saída prevista: [[0.08548134]]
Entrada: [0

In [ ]:
print('evaluating regression:')
evaluate_hyperparameters("regression")

evaluating regression:
Epoch 0, Loss: 0.11827800051206366
Epoch 100, Loss: 0.09245606985026665
Epoch 200, Loss: 0.09243716340338866
Epoch 300, Loss: 0.09240133767509828
Epoch 400, Loss: 0.09231568045927585
Epoch 500, Loss: 0.09199803031234458
Epoch 600, Loss: 0.08925099361664696
Epoch 700, Loss: 0.06737242404790304
Epoch 800, Loss: 0.010879609051634194
Epoch 900, Loss: 0.00353548570978942
Entrada: [-1.], Saída prevista: [[0.83182695]]
Entrada: [-0.97979798], Saída prevista: [[0.82446386]]
Entrada: [-0.95959596], Saída prevista: [[0.81597973]]
Entrada: [-0.93939394], Saída prevista: [[0.80622246]]
Entrada: [-0.91919192], Saída prevista: [[0.79502388]]
Epoch 0, Loss: 0.11353590371041195
Epoch 100, Loss: 0.02177547941516741
Epoch 200, Loss: 0.0015559837951129001
Epoch 300, Loss: 0.0014111615693224659
Epoch 400, Loss: 0.0013652706740250042
Epoch 500, Loss: 0.0013431057804273677
Epoch 600, Loss: 0.0013287868579507534
Epoch 700, Loss: 0.0013175122466649323
Epoch 800, Loss: 0.0013074507785854